# Deliverable 7.1: Nonlinear MPC for Landing

Implement and test a nonlinear MPC controller using CasADi for the full 12-state rocket system.

In [ ]:
%load_ext autoreload
%autoreload 2

# Get parent directory and add to sys.path
import sys, os
parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)

# Require ipympl
%matplotlib widget 

In [ ]:
from src.rocket import Rocket
from src.pos_rocket_vis import *
from LandMPC.nmpc_land import NmpcCtrl
import numpy as np

rocket_obj_path = os.path.join(parent_dir, "Cartoon_rocket.obj")
rocket_params_path = os.path.join(parent_dir, "rocket.yaml")

# Rocket setup
Ts  = 1/20
rocket = Rocket(Ts=Ts, model_params_filepath=rocket_params_path)
rocket.mass = 1.7  # Do not change!!!
rocket.controller_type = 'NmpcCtrl'  # IMPORTANT: Set for proper NMPC constraint checking

# Visualization setup
vis = RocketVis(rocket, rocket_obj_path)
vis.anim_rate = 1

In [ ]:
# Landing maneuver: from (3, 2, 10, 30°) to (1, 0, 3, 0°)
sim_time = 15  # simulation length in seconds

# Initial state: at (3, 2, 10, 30°)
x0 = np.array([0, 0, 0,  # angular velocities (wx, wy, wz)
               0, 0, np.deg2rad(30),  # angles (alpha, beta, gamma/roll)
               0, 0, 0,  # linear velocities (vx, vy, vz)
               3, 2, 10])  # positions (x, y, z)

# Target state: (1, 0, 3, 0°)
x_ref = np.array([0.]*9 + [1., 0., 3.])  # Target: x=1, y=0, z=3

# NMPC parameters - use longer horizon for large maneuvers
H = 4.0  # horizon time (increased from 2.0 for better long-term planning)

# Create nonlinear MPC controller (it will trim around x_ref internally)
nmpc = NmpcCtrl(rocket, Ts=Ts, H=H, x_ref=x_ref)
print("\nNonlinear MPC initialized")
print(f"  - Horizon: {nmpc.N} steps ({H}s)")
print(f"  - State dimension: {nmpc.nx}")
print(f"  - Input dimension: {nmpc.nu}")
print(f"  - Uses full nonlinear dynamics (no subsystem decomposition)")
print(f"  - Integration: RK4 (more accurate than Euler)")
print(f"  - Tuning: High terminal cost (P=10*Q) for convergence")

# Store trim point for later use
xs, us = nmpc.xs, nmpc.us

## Test 1: Check Open-Loop Trajectory

Before running closed-loop, verify that the optimal open-loop trajectory from the initial state is reasonable.

In [ ]:
# Compute open-loop optimal trajectory
u0, x_ol, u_ol, t_ol = nmpc.get_u(0.0, x0)

print("Open-loop trajectory computed:")
print(f"  - Initial control: {u0}")
print(f"  - Final state: {x_ol[:, -1]}")
print(f"  - Target state: {xs}")

# Plot open-loop trajectory using the standard plotting function
plot_static_states_inputs(t_ol[:-1], x_ol[:,:-1], u_ol, xs)

## Test 2: Closed-Loop Simulation

In [ ]:
t_cl, x_cl, u_cl, t_ol, x_ol, u_ol = rocket.simulate_land(nmpc, sim_time, H, x0)

## Diagnostics: Check NMPC Setup

Before simulating, let's verify the NMPC controller is working correctly.

In [ ]:

u_test, x_test, u_test_traj, t_test = nmpc.get_u(0.0, x0)


In [ ]:
vis.animate(t_cl[:-1], x_cl[:,:-1], u_cl, T_ol=t_ol[...,:-1], X_ol=x_ol, U_ol=u_ol)
plot_static_states_inputs(t_cl[:-1], x_cl[:,:-1], u_cl, xs)